In [1]:
import sevenbridges as sbg
from sevenbridges.errors import SbgError
import sys
from requests import request
import concurrent.futures
config = sbg.Config(profile='cavatica')
api = sbg.Api(config=config)

In [44]:
def get_inputs(api, project):
    tasks = api.tasks.query(project=project, status='COMPLETED').all()
    inputs = {}
    for task in tasks:
        # jointgenotyping-FM_01DTQB3Y
        parts = task.name.split('-')
        if parts[0] == 'jointgenotyping':
            sys.stderr.write('Found valid task: ' + task.name + '\n')
            fm_id = parts[1]
            try:
                ped = api.files.query(project=project, names=[(fm_id + '.ped')])[0]
                vqsr_vcf = task.outputs['finalgathervcf']
                inputs[fm_id] = {}
                inputs[fm_id]['ped'] =  ped
                inputs[fm_id]['vqsr_vcf'] = vqsr_vcf
            except:
                sys.stderr.write(task.name + ' has missing files, skipping\n')
    return inputs
            

In [45]:
def create_tasks(api, project, inputs, fm_id, suffix):
    cur_in = inputs[fm_id]
    task_name = 'kfdrc-peddy-' + fm_id + suffix
    app_name = project + '/kfdrc-peddy-tool'
    try:
        task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=cur_in, run=False)
        task.inputs['output_basename'] = task.id
        task.save()
        return 'FM_ID:' + fm_id + ' ' + inputs[fm_id]['vqsr_vcf'].name + ' Task:' + task.id
    except SbgError as e:
        sys.stderr.write(e.message + '; could not create task for ' + task_name + '!\n')

In [46]:
chung = 'kfdrc-harmonization/sd-46sk55a3-02'
schiff = 'kfdrc-harmonization/sd-ygva0e1c-01'
# cur_proj = schiff
out_fh = open('/Users/brownm28/Documents/2018-Oct-23_peddy_run/mara_rpt_b2_stage.txt', 'w')
# proj_dict = {'kfdrc-harmonization/sd-dypmehhf-01': 'Maris', 'kfdrc-harmonization/sd-9pyzahhe-08': 'Marazita_fy15_b1', 'kfdrc-harmonization/sd-9pyzahhe-09': 'Marazita_fy15_b2'}
proj_dict = {'kfdrc-harmonization/sd-9pyzahhe-09': 'Marazita_fy15_b2'}
suffix = '-rerun2'
for cur_proj in proj_dict:
    inputs = get_inputs(api, cur_proj)
    sys.stderr.write('Processing project ' + cur_proj + ': ' + proj_dict[cur_proj] + '\n')
    with concurrent.futures.ThreadPoolExecutor(8) as executor:
        results = {executor.submit(create_tasks, api, cur_proj, inputs, fm_id, suffix): fm_id for fm_id in inputs}
        for result in concurrent.futures.as_completed(results):
            out_fh.write(proj_dict[cur_proj] + ' ' + cur_proj + ' ' + result.result() + '\n')
out_fh.close()

Found valid task: jointgenotyping-FM_4P00M65H
Found valid task: jointgenotyping-FM_162NCRY3
Found valid task: jointgenotyping-FM_9E3WAF2Z
jointgenotyping-FM_9E3WAF2Z has missing files, skipping
Found valid task: jointgenotyping-FM_Q1JQJBB5
jointgenotyping-FM_Q1JQJBB5 has missing files, skipping
Found valid task: jointgenotyping-FM_MA2B7XP5
Found valid task: jointgenotyping-FM_34H49TGW
Found valid task: jointgenotyping-FM_3PMTR62F
Found valid task: jointgenotyping-FM_V7CNVFHM
Found valid task: jointgenotyping-FM_D5C8910K
Found valid task: jointgenotyping-FM_QGQ6QZ2X
Found valid task: jointgenotyping-FM_BWKVDB1G
Found valid task: jointgenotyping-FM_H69YN234
Found valid task: jointgenotyping-FM_ER8NTQ3P
Found valid task: jointgenotyping-FM_9WAHMGRV
Found valid task: jointgenotyping-FM_WR4R70NY
Found valid task: jointgenotyping-FM_JN9AW927
Found valid task: jointgenotyping-FM_3FJ746JF
Found valid task: jointgenotyping-FM_QEH6QM74
Found valid task: jointgenotyping-FM_PF276JVF
jointgenotypin

In [47]:
def run_task(api, entry):
    info = entry.rstrip('\n').split()
    (tag, tid) = info[-1].split(':')
    task = api.tasks.get(id=tid)
    task.run()

In [48]:

task_fn = '/Users/brownm28/Documents/2018-Oct-23_peddy_run/mara_rpt_b2_stage.txt'
with concurrent.futures.ThreadPoolExecutor(8) as executor:
    results = {executor.submit(run_task, api, entry): entry for entry in open(task_fn)}


In [5]:
def rm_outputs(api, project, prefix_list, fm_list):
    tasks = api.tasks.query(project=project, status='COMPLETED').all()
    for task in tasks:
        # jointgenotyping-FM_01DTQB3Y
        parts = task.name.split('-')
        if parts[0] in prefix_list and parts[1] in fm_list:
            sys.stderr.write('Found task to remove outputs from: ' + task.name + '\n')
            for output in task.outputs:
                if task.outputs[output] is None:
                    sys.stderr.write('Skipping None object for ' + output + '\n')
                elif not isinstance(task.outputs[output], list):
                    try:
                        sys.stderr.write('Deleting output ' + task.outputs[output].name + '\n')
                        task.outputs[output].delete()
                    except:
                        sys.stderr.write('File for output ' + output + ' does not exist, cannot delete\n')
                else:
                    for obj in task.outputs[output]:
                        sys.stderr.write('Deleting output ' + obj.name + '\n')
                        obj.delete()
    

In [6]:
prefix_list = ['kf_genotype_refinement_vep']
project = 'kfdrc-harmonization/sd-9pyzahhe-08'
fm_list = []
fm_file = open('/Users/brownm28/Documents/2018-Oct-9_genotype_refinement_bonanza/MARAZITA/batch1_fm_rpt.txt')
for line in fm_file:
    fm_list.append(line.rstrip('\n'))
fm_file.close()
rm_outputs(api, project, prefix_list, fm_list)

Found task to remove outputs from: kf_genotype_refinement_vep-FM_3YSAERT6
Skipping None object for vep_warn
File for output cgp_vep_annotated_vcf does not exist, cannot delete
File for output vcf_summary_stats does not exist, cannot delete
Found task to remove outputs from: kf_genotype_refinement_vep-FM_4G4QGRH1
Skipping None object for vep_warn
File for output cgp_vep_annotated_vcf does not exist, cannot delete
File for output vcf_summary_stats does not exist, cannot delete
Found task to remove outputs from: kf_genotype_refinement_vep-FM_54AAR0KH
Skipping None object for vep_warn
Deleting output 6e1d023e-d9fe-4dcb-9504-edbe247b70b3.CGP.filtered.deNovo.vep.vcf.gz
Deleting output 6e1d023e-d9fe-4dcb-9504-edbe247b70b3_stats.txt
Found task to remove outputs from: kf_genotype_refinement_vep-FM_7Y0MZVCX
Skipping None object for vep_warn
Deleting output 725308b0-630d-4de9-baba-0f83687ed83a.CGP.filtered.deNovo.vep.vcf.gz
Deleting output 725308b0-630d-4de9-baba-0f83687ed83a_stats.txt
Found task

Deleting output 0918148a-5870-43c4-bc79-6883c7675cad_stats.txt
Found task to remove outputs from: kf_genotype_refinement_vep-FM_SA6Y47WQ
Skipping None object for vep_warn
Deleting output af19f476-f44b-4de5-b5d4-94619c14cd25.CGP.filtered.deNovo.vep.vcf.gz
Deleting output af19f476-f44b-4de5-b5d4-94619c14cd25_stats.txt
Found task to remove outputs from: kf_genotype_refinement_vep-FM_SN2BRZTM
Skipping None object for vep_warn
Deleting output 8d1ea1ae-ee52-44d8-a3e4-13511500b0a9.CGP.filtered.deNovo.vep.vcf.gz
Deleting output 8d1ea1ae-ee52-44d8-a3e4-13511500b0a9_stats.txt
Found task to remove outputs from: kf_genotype_refinement_vep-FM_1PEC3KD8
Skipping None object for vep_warn
Deleting output bd7925b9-b27e-4b85-a39e-cc4b11316848.CGP.filtered.deNovo.vep.vcf.gz
Deleting output bd7925b9-b27e-4b85-a39e-cc4b11316848_stats.txt
Found task to remove outputs from: kf_genotype_refinement_vep-FM_4Y21X6PP
Skipping None object for vep_warn
Deleting output 9256b345-e778-43a0-baae-de33efc83160.CGP.filtere